<a href="https://colab.research.google.com/github/arwakhalidw/2024-GP-Group6/blob/main/AlzheimerPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import required libraries
import pandas as pd
import numpy as np

In [ ]:
# read the dataset
dataset = pd.read_csv("alzheimer.csv")
# copy the dataset
data = dataset.copy() # to keep the original dataset unchanged

there are 9 features, they are as follows:
1.  Group is the Class lable
2.  Age is the Age of the patient
3.  EDUC are Years of Education
4.  SES are Socioeconomic Status on a scale from 1 to 5
5.  MMSE is Mini Mental State Examination
6.  CDR is Clinical Dementia Rating
7.  eTIV is Estimated total intracranial volume
8.  nWBV is Normalize Whole Brain Volume
9.  ASF is Atlas Scaling Factor





